<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/Hill_Climbing_vs_Recocido_Simulado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

## Definición de función Alpine.

In [ ]:
def alpine(x, y):
    """Calcula el valor de la funcion Alpine en 2D."""
    return np.abs(x * np.sin(x) + 0.1 * x) + np.abs(y * np.sin(y) + 0.1 * y)

## Implementación de recocido simulado.

In [ ]:
def recocido_simulado(temp_inicial, temp_final, factor_enfriamiento,
                      iteraciones_por_temp):
    """
    Algoritmo de recocido simulado para minimizar la funcion Alpine.
    """
    # Generar solucion inicial aleatoria en [-5, 5] x [-5, 5]
    x_actual = random.uniform(-5, 5)
    y_actual = random.uniform(-5, 5)
    costo_actual = alpine(x_actual, y_actual)

    # Inicializar mejor solucion encontrada
    mejor_x = x_actual
    mejor_y = y_actual
    mejor_costo = costo_actual

    # Almacenar historial para visualizacion posterior
    historial_x = [x_actual]
    historial_y = [y_actual]
    historial_costos = [costo_actual]

    # Establecer temperatura inicial
    temperatura = temp_inicial

    # Bucle externo: proceso de enfriamiento
    while temperatura > temp_final:

        # Estadisticas para esta temperatura (opcional)
        aceptaciones = 0
        mejoras = 0

        # Bucle interno: iteraciones a temperatura constante
        for _ in range(iteraciones_por_temp):

            # Generar solucion vecina con perturbacion pequena
            dx = random.uniform(-0.5, 0.5)
            dy = random.uniform(-0.5, 0.5)
            x_vecino = x_actual + dx
            y_vecino = y_actual + dy

            # Aplicar limites con reflexion en [-5, 5]
            if x_vecino < -5.0:
                x_vecino = -5.0 - (x_vecino + 5.0)
            elif x_vecino > 5.0:
                x_vecino = 5.0 - (x_vecino - 5.0)

            if y_vecino < -5.0:
                y_vecino = -5.0 - (y_vecino + 5.0)
            elif y_vecino > 5.0:
                y_vecino = 5.0 - (y_vecino - 5.0)

            # Evaluar costo de la solucion vecina
            costo_vecino = alpine(x_vecino, y_vecino)

            # Calcular diferencia de costo (delta)
            delta_costo = costo_vecino - costo_actual

            # Criterio de aceptacion de Metropolis
            if delta_costo <= 0:
                # Mejora: aceptar siempre
                aceptar = True
                mejoras += 1
            else:
                # Empeoramiento: aceptar con probabilidad de Boltzmann
                probabilidad = math.exp(-delta_costo / temperatura)
                aceptar = random.random() < probabilidad

            # Aplicar decision de aceptacion
            if aceptar:
                x_actual = x_vecino
                y_actual = y_vecino
                costo_actual = costo_vecino
                aceptaciones += 1

                # Actualizar mejor solucion si es necesario
                if costo_actual < mejor_costo:
                    mejor_x = x_actual
                    mejor_y = y_actual
                    mejor_costo = costo_actual
        # Registrar estado al final de este nivel de temperatura
        # (solo guardamos cuando cambia la temperatura para no saturar memoria)
        historial_x.append(x_actual)
        historial_y.append(y_actual)
        historial_costos.append(costo_actual)

        # Esquema de enfriamiento geometrico
        temperatura = temperatura * factor_enfriamiento

    # Devolver mejor solucion encontrada e historial
    return mejor_x, mejor_y, mejor_costo, historial_x, historial_y, historial_costos

## Implementación de hill climbing.

In [ ]:
def hill_climbing(x_inicial, y_inicial, tam_paso, num_vec, num_iteraciones):
    x_actual = x_inicial
    y_actual = y_inicial
    valor_actual = alpine(x_actual, y_actual)

    # Guardar las coordenadas iniciales
    # para graficar el comportamiento del algoritmo
    coordenadas_x = [x_actual]
    coordenadas_y = [y_actual]
    valores = [valor_actual]

    # Inicializar contador antes del bucle para evitar UnboundLocalError
    contador = 0

    for i in range(num_iteraciones):
        # Generar vecinos
        vecinos = []
        for dx in range(num_vec):
            dx = random.uniform(-tam_paso, tam_paso)
            dy = random.uniform(-tam_paso, tam_paso)
            x = x_actual + dx
            y = y_actual + dy

            # Revisar que los valores estén en [-5, 5]
            if x < -5.0:
              x = -5 - (x + 5)
            if x > 5.0:
              x = 5 - (x - 5)
            if y < -5.0:
              y = -5 - (y + 5)
            if y > 5.0:
              y = 5 - (y - 5)
            vecinos.append((x, y))

        # Evaluar vecinos y encontrar el mejor vecino
        mejor_valor_vecino = float('inf')
        mejor_vecino = None
        for vecino_x, vecino_y in vecinos:
            valor_vecino = alpine(vecino_x, vecino_y)
            if valor_vecino < mejor_valor_vecino:
                mejor_valor_vecino = valor_vecino
                mejor_vecino = (vecino_x, vecino_y)

        # Actualizar la posición actual
        if mejor_valor_vecino < valor_actual:
            x_actual, y_actual = mejor_vecino
            valor_actual = mejor_valor_vecino

            # Guardamos las coordenadas y el costo
            # para graficar el comportamiento del algoritmo
            coordenadas_x.append(x_actual)
            coordenadas_y.append(y_actual)
            valores.append(valor_actual)
            contador = 0
        else:
            contador += 1
            if contador >= 100:
                break

    return coordenadas_x, coordenadas_y, valores

## Diseño del experimento.

In [ ]:
# Configuracion del experimento
num_ejecuciones = 30
np.random.seed(42)  # Para reproducibilidad

# Configuracion de recocido simulado
config_recocido = {
    'temp_inicial': 100.0,
    'temp_final': 0.01,
    'factor_enfriamiento': 0.95,
    'iteraciones_por_temp': 100
}

# Configuracion de hill climbing (del capitulo anterior)
config_hill = {
    'tam_paso': 0.5,
    'num_vecinos': 20,
    'num_iteraciones': 100
}

# Almacenar resultados
costos_recocido = []
costos_hill = []
distancias_recocido = []
distancias_hill = []

print("Ejecutando experimento comparativo...")
print("="*60)

for i in range(num_ejecuciones):
    # Punto inicial aleatorio (mismo para ambos algoritmos)
    x_ini = random.uniform(-5, 5)
    y_ini = random.uniform(-5, 5)

    # Ejecutar recocido simulado
    mejor_x_r, mejor_y_r, mejor_costo_r, _, _, _ = recocido_simulado(
        config_recocido['temp_inicial'],
        config_recocido['temp_final'],
        config_recocido['factor_enfriamiento'],
        config_recocido['iteraciones_por_temp']
    )

    # Ejecutar hill climbing (asumiendo que tenemos la funcion del cap anterior)
    hist_x_h, hist_y_h, hist_costos_h = hill_climbing(
        x_ini, y_ini,
        config_hill['tam_paso'],
        config_hill['num_vecinos'],
        config_hill['num_iteraciones']
    )
    mejor_x_h = hist_x_h[-1]
    mejor_y_h = hist_y_h[-1]
    mejor_costo_h = hist_costos_h[-1]

    # Guardar resultados
    costos_recocido.append(mejor_costo_r)
    costos_hill.append(mejor_costo_h)

    distancias_recocido.append(np.sqrt(mejor_x_r**2 + mejor_y_r**2))
    distancias_hill.append(np.sqrt(mejor_x_h**2 + mejor_y_h**2))

    # Mostrar progreso
    if (i + 1) % 10 == 0:
        print(f"Completadas {i + 1}/{num_ejecuciones} ejecuciones")

print("\nExperimento completado!")

## Análisis estadístico de los resultados.

In [ ]:
# Calcular estadisticos para recocido simulado
print("="*70)
print("ANALISIS ESTADISTICO COMPARATIVO")
print("="*70)

print("\n" + "="*70)
print("RECOCIDO SIMULADO")
print("="*70)
print(f"Costos encontrados:")
print(f"  Media:              {np.mean(costos_recocido):.6f}")
print(f"  Mediana:            {np.median(costos_recocido):.6f}")
print(f"  Desviacion estandar: {np.std(costos_recocido, ddof=1):.6f}")
print(f"  Minimo:             {np.min(costos_recocido):.6f}")
print(f"  Maximo:             {np.max(costos_recocido):.6f}")

cuartiles_r = np.percentile(costos_recocido, [25, 50, 75])
print(f"  Cuartiles (Q1,Q2,Q3): [{cuartiles_r[0]:.4f}, {cuartiles_r[1]:.4f}, {cuartiles_r[2]:.4f}]")

# Contar exitos (costo < 0.1 se considera encontrar el optimo)
exitos_r = sum(1 for c in costos_recocido if c < 0.1)
print(f"\nEjecuciones exitosas (costo < 0.1): {exitos_r}/30 ({exitos_r/30*100:.1f}%)")

print("\n" + "="*70)
print("HILL CLIMBING")
print("="*70)
print(f"Costos encontrados:")
print(f"  Media:              {np.mean(costos_hill):.6f}")
print(f"  Mediana:            {np.median(costos_hill):.6f}")
print(f"  Desviacion estandar: {np.std(costos_hill, ddof=1):.6f}")
print(f"  Minimo:             {np.min(costos_hill):.6f}")
print(f"  Maximo:             {np.max(costos_hill):.6f}")

cuartiles_h = np.percentile(costos_hill, [25, 50, 75])
print(f"  Cuartiles (Q1,Q2,Q3): [{cuartiles_h[0]:.4f}, {cuartiles_h[1]:.4f}, {cuartiles_h[2]:.4f}]")

exitos_h = sum(1 for c in costos_hill if c < 0.1)
print(f"\nEjecuciones exitosas (costo < 0.1): {exitos_h}/30 ({exitos_h/30*100:.1f}%)")

# Comparacion directa
print("\n" + "="*70)
print("COMPARACION")
print("="*70)
mejoria_media = ((np.mean(costos_hill) - np.mean(costos_recocido)) /
                 np.mean(costos_hill) * 100)
print(f"Mejoria en costo medio: {mejoria_media:.2f}%")
print(f"Reduccion en desviacion estandar: {(np.std(costos_hill, ddof=1) - np.std(costos_recocido, ddof=1)):.6f}")

## Creamos diagramas de cajas

In [ ]:
# Crear diagrama de cajas
datos = [costos_hill, costos_recocido]
etiquetas = ['Hill Climbing', 'Recocido Simulado']

plt.figure(figsize=(10, 7))

# Crear boxplot
bp = plt.boxplot(datos,
                 labels=etiquetas,
                 patch_artist=True,
                 widths=0.6)

# Personalizar colores
colores = ['lightcoral', 'lightblue']
for patch, color in zip(bp['boxes'], colores):
    patch.set_facecolor(color)

# Linea del optimo global
plt.axhline(y=0, color='green', linestyle='--', linewidth=2,
            label='Optimo global', alpha=0.7)

# Configuracion
plt.ylabel('Costo de la solucion f(x,y)', fontsize=12)
plt.title('Comparacion de algoritmos: Hill Climbing vs Recocido Simulado\n(30 ejecuciones cada uno)',
          fontsize=14)
plt.grid(True, alpha=0.3, axis='y')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()